# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [39]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [40]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [41]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [61]:
df=data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": "sum"}).reset_index()
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [51]:
pivot=df.pivot_table(values="Quantity", index="ProductName", columns="CustomerID").fillna(0).T

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [52]:
def create_similarity(dataframe, metric='euclidean'):
    """
    Given a dataframe, return its similarity matrix 
    """
    distance_vector = pdist(dataframe, metric)
    distance_matrix = squareform(distance_vector)
    similarities = 1 / (1 + distance_matrix)
    
    similarity_index = pd.DataFrame(similarities, 
                                    index=dataframe.index, 
                                    columns=dataframe.index)
    return similarity_index

In [53]:
new_data=create_similarity(pivot)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [54]:
lista=list(new_data[33].sort_values(ascending=False).head(6).drop(33).index)
lista

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [68]:
df1=df[df["CustomerID"].isin(lista)]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [71]:
df2=df1.groupby(by="ProductName").agg({"Quantity": "sum"}).reset_index().sort_values("Quantity", ascending=False)

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [75]:
df3=df2.merge(pivot.T, on="ProductName")


,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,50.0,0.0,25.0,0.0,0.0,0.0,0.0
2,Towels - Paper / Kraft,3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Soup - Campbells Bean Medley,3,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Hinge W Undercut,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0
214,Ice Cream Bar - Hageen Daz To,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0
215,Jagermeister,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
216,Jolt Cola - Electric Blue,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.